In [11]:
import sys
sys.path.append('/home/andrew/Documents/OOI-CGSN/')

In [29]:
import requests

In [15]:
from QAQC_Sandbox.Metadata_Review.utils import *

In [16]:
# Try building a function to do the file path generator
def generate_file_path(dirpath,filename,ext=['.cap','.txt','.log'],exclude=['_V','_Data_Workshop']):
    """
    Function which searches for the location of the given file and returns
    the full path to the file.
    
    Args:
        dirpath - parent directory path under which to search
        filename - the name of the file to search for
        ext - 
        exclude - optional list which allows for excluding certain
            directories from the search
    Returns:
        fpath - the file path to the filename from the current
            working directory.
    """
    for root, dirs, files in os.walk(dirpath):
        dirs[:] = [d for d in dirs if d not in exclude]
        for fname in files:
            if fnmatch.fnmatch(fname, [filename+'*'+x for x in ext]):
                fpath = os.path.join(root, fname)
                return fpath

In [17]:
def ensure_dir(file_path):
    """
    Function which checks that the directory where you want
    to save a file exists. If it doesn't, it creates the 
    directory.
    """
    if not os.path.exists(file_path):
        os.makedirs(file_path)

In [18]:
qct_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/'
cal_directory = '/media/andrew/OS/Users/areed/Documents/Project_Files/Records/Instrument_Records/OPTAA/'
asset_management_directory = '/home/andrew/Documents/OOI-CGSN/ooi-integration/asset-management/calibration/OPTAA/'

In [19]:
#excel_spreadsheet = 'C:/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
excel_spreadsheet = '/media/andrew/OS/Users/areed/Documents/Project_Files/Documentation/System/System Notebook/WHOI_Asset_Tracking.xlsx'
sheet_name = 'Sensors'

In [20]:
OPTAA = whoi_asset_tracking(excel_spreadsheet,sheet_name,instrument_class='OPTAA')

## Assign the Stream URLs

In [22]:
data_url = 'https://ooinet.oceanobservatories.org/api/m2m/12576/sensor/inv'
anno_url = 'https://ooinet.oceanobservatories.org/api/m2m/12580/anno/find'
vocab_url = 'https://ooinet.oceanobservatories.org/api/m2m/12586/vocab/inv'
asset_url = 'https://ooinet.oceanobservatories.org/api/m2m/12587'
deploy_url = asset_url + '/events/deployment/query'
cal_url = asset_url + '/asset/cal'

In [27]:
username = 'OOIAPI-C9OSZAQABG1H3U'
token = 'JA48WUQVG7F'

In [226]:
# List the site, node, instrument names
site = 'CP03ISSM'
node = 'RID27'
sensor = '03-CTDBPC000'
method = 'recovered_host' # 'recovered_inst' 'telemetered'

In [227]:
# Function to make an API request and print the results
def get_and_print_api(url):
    r = requests.get(url, auth=(username, token))
    data = r.json()
    for d in data:
        print(d)

In [228]:
# Specify some functions to convert timestamps
ntp_epoch = datetime.datetime(1900, 1, 1)
unix_epoch = datetime.datetime(1970, 1, 1)
ntp_delta = (unix_epoch - ntp_epoch).total_seconds()

def ntp_seconds_to_datetime(ntp_seconds):
    return datetime.datetime.utcfromtimestamp(ntp_seconds - ntp_delta).replace(microsecond=0)
  
def convert_time(ms):
    if ms is None:
        return None
    else:
        return datetime.datetime.utcfromtimestamp(ms/1000)

In [229]:
get_and_print_api('/'.join((data_url,site,'RID27','03-CTDBPC000','recovered_host')))

ctdbp_cdef_dcl_instrument_recovered


In [230]:
# Retrieve vocabulary information for a given instrument
request_url = '/'.join((vocab_url, site, node, sensor))
r = requests.get(request_url, auth=(username, token))
data = r.json()

In [231]:
data

[{'@class': '.VocabRecord',
  'model': 'SBE 16plusV2',
  'manufacturer': 'Sea-Bird',
  'tocL3': 'Near Surface Instrument Frame',
  'mindepth': 7.0,
  'maxdepth': 7.0,
  'vocabId': 583,
  'refdes': 'CP03ISSM-RID27-03-CTDBPC000',
  'instrument': 'CTD',
  'tocL1': 'Coastal Pioneer',
  'tocL2': 'Inshore Surface Mooring'}]

In [232]:
refdes = data[0]['refdes']
refdes

'CP03ISSM-RID27-03-CTDBPC000'

In [233]:
# Set up the API request
cal_request_url = cal_url
params = {
    'refdes':refdes,
}
# Get the information from the server
r = requests.get(cal_request_url, params=params, auth=(username, token))
cal_data = r.json()

In [234]:
def reformat_cal_data(cal_data):
    df = pd.DataFrame()
    for d in cal_data:
        for dd in d['sensor']['calibration']:
            for ddd in dd['calData']:
                df = df.append({
                    'value': ddd['value'],
                    'start': convert_time(ddd['eventStartTime']),
                    'stop': convert_time(ddd['eventStopTime']),
                    'name': ddd['eventName'],
                    'assetUid': ddd['assetUid'],
                    'calSheet': ddd['dataSource'],
                }, ignore_index=True)
    df = df.sort_values(by=['start','name'])
    return df

In [235]:
cal_df = reformat_cal_data(cal_data)
cal_df.head(10)

,assetUid,calSheet,name,start,stop,value
465,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a0,2011-05-18,None,1.301118e-03
729,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a0,2011-05-18,None,1.301118e-03
498,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a1,2011-05-18,None,2.561486e-04
762,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a1,2011-05-18,None,2.561486e-04
493,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a2,2011-05-18,None,1.896537e-07
757,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a2,2011-05-18,None,1.896537e-07
489,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a3,2011-05-18,None,1.301189e-07
753,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_a3,2011-05-18,None,1.301189e-07
520,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_cpcor,2011-05-18,None,-9.570000e-08
784,CGINS-CTDBPC-06841,CGINS-CTDBPC-06841__20110518_Cal_Info.xlsx,CC_cpcor,2011-05-18,None,-9.570000e-08


## Check the deployment history of the streams

In [236]:
# Set up the API request url
deploy_request_url = deploy_url
params = {
    'refdes':refdes,
}

# Get the information from the server
r = requests.get(deploy_request_url, params=params, auth=(username, token))
deploy_data = r.json()

In [237]:
deploy_data

[{'@class': '.XDeployment',
  'location': {'location': [-70.87862, 40.36222],
   'depth': 7.0,
   'latitude': 40.36222,
   'longitude': -70.87862,
   'orbitRadius': None},
  'editPhase': 'OPERATIONAL',
  'node': None,
  'referenceDesignator': 'CP03ISSM-RID27-03-CTDBPC000',
  'sensor': {'@class': '.XInstrument',
   'calibration': [{'@class': '.XCalibration',
     'name': 'CC_a0',
     'calData': [{'@class': '.XCalibrationData',
       'value': 0.001273664,
       'comments': None,
       'assetUid': 'CGINS-CTDBPC-50003',
       'eventId': 11497,
       'eventType': 'CALIBRATION_DATA',
       'eventName': 'CC_a0',
       'eventStartTime': 1481932800000,
       'eventStopTime': None,
       'notes': None,
       'tense': 'UNKNOWN',
       'dataSource': 'CGINS-CTDBPC-50003__20161217_Cal_Info.xlsx',
       'lastModifiedTimestamp': 1550150025777},
      {'@class': '.XCalibrationData',
       'value': 0.001268488,
       'comments': None,
       'assetUid': 'CGINS-CTDBPC-50003',
       'event

In [238]:
df = pd.DataFrame()
for d in deploy_data:
    for i in range(0,len(d['sensor']['calibration'])):
        dn = d['deploymentNumber']
        uid = d['sensor']['uid']    
        name = d['sensor']['calibration'][i].get('name')
        cc_uid = d['sensor']['calibration'][i]['calData'][0].get('assetUid')
        values = d['sensor']['calibration'][i]['calData'][0].get('value')
        source = d['sensor']['calibration'][i]['calData'][0].get('dataSource')
        # Now make a dataframe of the results
        df = df.append({
            'Deployment':dn,
            'UID':uid,
            'CC_UID':cc_uid,
            'CC_name':name,
            'CC_values':values,
            'CC_source':source
        }, ignore_index=True)
    

In [239]:
df.set_index('Deployment',inplace=True)

In [240]:
df

,CC_UID,CC_name,CC_source,CC_values,UID
Deployment,,,,,
1.0,CGINS-CTDBPC-50003,CC_a0,CGINS-CTDBPC-50003__20161217_Cal_Info.xlsx,1.273664e-03,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_ptcb1,CGINS-CTDBPC-50003__20140130_Cal_Info.xlsx,-9.000000e-04,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_g,CGINS-CTDBPC-50003__20161217_Cal_Info.xlsx,-9.785720e-01,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_ptcb2,CGINS-CTDBPC-50003__20161217_Cal_Info.xlsx,0.000000e+00,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_h,CGINS-CTDBPC-50003__20171212_Cal_Info.xlsx,1.366856e-01,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_i,CGINS-CTDBPC-50003__20171212_Cal_Info.xlsx,-2.315591e-04,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_a3,CGINS-CTDBPC-50003__20161217_Cal_Info.xlsx,1.592651e-07,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_a2,CGINS-CTDBPC-50003__20140130_Cal_Info.xlsx,-4.940714e-07,CGINS-CTDBPC-50003
1.0,CGINS-CTDBPC-50003,CC_ptcb0,CGINS-CTDBPC-50003__20151215_Cal_Info.xlsx,2.502750e+01,CGINS-CTDBPC-50003


In [182]:
df.to_csv('Ticket 9184: Deployment_and_calibration_sheets.csv')

In [67]:
def reformat_deployment_data(deploy_data):
    df = pd.DataFrame()
    for d in deploy_data:
        df = df.append({
            'deployment': d['deploymentNumber'],
            'start': convert_time(d['eventStartTime']),
            'stop': convert_time(d['eventStopTime']),
            'latitude': d['location']['latitude'],
            'longitude': d['location']['longitude'],
            'sensor': d['sensor']['assetId'],
        }, ignore_index=True)
    return df

In [68]:
deploy_df = reformat_deployment_data(deploy_data)
deploy_df

,deployment,latitude,longitude,sensor,start,stop
0,1.0,40.136780,-70.76978,511.0,2013-11-21 18:15:00,2014-04-18 00:00:00
1,2.0,40.133900,-70.77890,498.0,2014-12-13 18:47:00,2014-12-15 20:58:00
2,3.0,40.140220,-70.77128,498.0,2015-05-07 17:34:00,2015-10-23 19:40:00
3,4.0,40.133230,-70.77843,497.0,2015-10-23 18:49:00,2016-04-04 12:03:00
4,5.0,40.140370,-70.77133,503.0,2016-05-13 13:50:00,2016-10-13 19:34:00
5,6.0,40.133420,-70.77847,764.0,2016-10-13 18:36:00,2017-06-09 16:05:00
6,7.0,40.139817,-70.77115,511.0,2017-06-09 14:24:00,2017-11-01 20:33:00
7,8.0,40.133383,-70.77830,451.0,2017-10-29 14:15:00,2018-03-29 19:37:00
8,9.0,40.139750,-70.77128,498.0,2018-03-24 21:32:00,2018-10-29 12:31:00
9,10.0,40.133367,-70.77770,745.0,2018-10-30 01:48:00,NaT


Check that the calibration sheet dates match the calibration sheet dates

In [177]:
cal_sheets = {}
for ind in np.unique(df.index.values):
    cal_sheets.update({ind:np.unique(df.loc[ind]['CC_source'])})

In [185]:
df2 = pd.DataFrame()
for key in cal_sheets.keys():
    df2 = df2.append({
        'Deployment':key,
        'Calibration Sheets':cal_sheets.get(key)
    }, ignore_index = True)

In [191]:
df2

,Calibration Sheets,Deployment
0,"[CGINS-OPTAAD-00123__20131121_Cal_Info.xlsx, C...",1.0
1,"[CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx, C...",2.0
2,"[CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx, C...",3.0
3,[CGINS-OPTAAD-00152__20151023_Cal_Info.xlsx],4.0
4,"[CGINS-OPTAAD-00129__20131121_Cal_Info.xlsx, C...",5.0
5,[CGINS-OPTAAD-00207__20150315_Cal_Info.xlsx],6.0
6,"[CGINS-OPTAAD-00123__20131121_Cal_Info.xlsx, C...",7.0
7,"[CGINS-OPTAAD-00185__20141213_Cal_Info.xlsx, C...",8.0
8,"[CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx, C...",9.0
9,[CGINS-OPTAAD-00222__20150815_Cal_Info.xlsx],10.0


In [175]:
ind = np.unique(df.index.values)[0]

In [179]:
df.loc[2.0]

,CC_UID,CC_name,CC_source,CC_values,UID
Deployment,,,,,
2.0,CGINS-OPTAAD-00151,CC_awlngth,CGINS-OPTAAD-00151__20160528_Cal_Info.xlsx,"[400.0, 403.4, 407.5, 410.7, 414.2, 417.8, 422...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_tbins,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx,"[0.975714, 1.356792, 2.427021, 3.485345, 4.504...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_ccwo,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx,"[0.323797, 0.411719, 0.498724, 0.564484, 0.625...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_taarray,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx,"[[-0.03019, -0.02992, -0.030121, -0.028327, -0...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_acwo,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx,"[-0.829099, -0.58337, -0.394929, -0.260448, -0...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_tcarray,CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx,"[[0.072521, 0.070957, 0.063394, 0.060146000000...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_cwlngth,CGINS-OPTAAD-00151__20160528_Cal_Info.xlsx,"[400.1, 404.1, 407.5, 410.7, 414.2, 418.0, 422...",CGINS-OPTAAD-00151
2.0,CGINS-OPTAAD-00151,CC_tcal,CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx,20.2,CGINS-OPTAAD-00151


In [192]:
def explode(df, lst_cols, fill_value='', preserve_index=False):
    # make sure `lst_cols` is list-alike
    if (lst_cols is not None
        and len(lst_cols) > 0
        and not isinstance(lst_cols, (list, tuple, np.ndarray, pd.Series))):
        lst_cols = [lst_cols]
    # all columns except `lst_cols`
    idx_cols = df.columns.difference(lst_cols)
    # calculate lengths of lists
    lens = df[lst_cols[0]].str.len()
    # preserve original index values    
    idx = np.repeat(df.index.values, lens)
    # create "exploded" DF
    res = (pd.DataFrame({
                col:np.repeat(df[col].values, lens)
                for col in idx_cols},
                index=idx)
             .assign(**{col:np.concatenate(df.loc[lens>0, col].values)
                            for col in lst_cols}))
    # append those rows that have empty lists
    if (lens == 0).any():
        # at least one list in cells is empty
        res = (res.append(df.loc[lens==0, idx_cols], sort=False)
                  .fillna(fill_value))
    # revert the original index order
    res = res.sort_index()
    # reset index if requested
    if not preserve_index:        
        res = res.reset_index(drop=True)
    return res

In [196]:
df2 = explode(df2, ['Calibration Sheets'], fill_value='', preserve_index=True)

In [197]:
df2

,Deployment,Calibration Sheets
0,1.0,CGINS-OPTAAD-00123__20131121_Cal_Info.xlsx
0,1.0,CGINS-OPTAAD-00123__20151021_Cal_Info.xlsx
1,2.0,CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx
1,2.0,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx
1,2.0,CGINS-OPTAAD-00151__20160528_Cal_Info.xlsx
2,3.0,CGINS-OPTAAD-00151__20141213_Cal_Info.xlsx
2,3.0,CGINS-OPTAAD-00151__20150507_Cal_Info.xlsx
2,3.0,CGINS-OPTAAD-00151__20160528_Cal_Info.xlsx
3,4.0,CGINS-OPTAAD-00152__20151023_Cal_Info.xlsx
4,5.0,CGINS-OPTAAD-00129__20131121_Cal_Info.xlsx


In [198]:
df2.to_csv('Ticket 9184: Deployment and Calibration Sheet List.csv')

In [247]:
# Get deployment digests by the unique asset identifier (UID).
ASSET_BASE_URL = 'https://ooinet.oceanobservatories.org/api/m2m/12587/asset'
request_url = ASSET_BASE_URL+'/cal?refdes=GP03FLMB-RIM01-02-CTDMOG060'

In [245]:
stuff

In [250]:
r = requests.get(request_url, auth=(username, token))
data = r.json()

# print( json.dumps(data, indent=4) )
# data[4]
df3 = pd.DataFrame(data)
df3

,@class,assetUid,dataSource,deployCruiseInfo,deployedBy,deploymentNumber,editPhase,eventId,eventName,eventStartTime,eventStopTime,eventType,inductiveId,ingestInfo,lastModifiedTimestamp,location,mooring,node,notes,recoverCruiseInfo,recoveredBy,referenceDesignator,sensor,tense,versionNumber,waterDepth
0,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,1,OPERATIONAL,4036,GP03FLMB-RIM01-02-CTDMOG060,1374647520000,1.403050e+12,DEPLOYMENT,None,[],1550149962206,"{'location': [-144.40083, 50.33167], 'depth': ...","{'@class': '.XMooring', 'assetId': 150, 'remot...",None,None,"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4145.0
1,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,2,OPERATIONAL,4054,GP03FLMB-RIM01-02-CTDMOG060,1403229600000,1.433635e+12,DEPLOYMENT,None,[],1550149962381,"{'location': [-144.3975, 50.33133], 'depth': 3...","{'@class': '.XMooring', 'assetId': 152, 'remot...",None,None,"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4145.0
2,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,3,OPERATIONAL,4072,GP03FLMB-RIM01-02-CTDMOG060,1433798700000,1.467571e+12,DEPLOYMENT,None,[],1550149962559,"{'location': [-144.39833, 50.33033], 'depth': ...","{'@class': '.XMooring', 'assetId': 154, 'remot...",None,None,"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4145.0
3,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,4,OPERATIONAL,4090,GP03FLMB-RIM01-02-CTDMOG060,1467673620000,1.500304e+12,DEPLOYMENT,None,[],1550149962730,"{'location': [-144.398, 50.32925], 'depth': No...","{'@class': '.XMooring', 'assetId': 164, 'remot...",None,None,"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4146.0
4,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,5,OPERATIONAL,4108,GP03FLMB-RIM01-02-CTDMOG060,1500072540000,1.532539e+12,DEPLOYMENT,None,[],1550149962895,"{'location': [-144.514867, 50.37768], 'depth':...","{'@class': '.XMooring', 'assetId': 162, 'remot...",None,None,"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4169.0
5,.XDeployment,None,Load from [GP03FLMB_Deploy.xlsx],"{'@class': '.CruiseInfo', 'uniqueCruiseIdentif...",None,6,OPERATIONAL,4126,GP03FLMB-RIM01-02-CTDMOG060,1532473260000,NaN,DEPLOYMENT,None,[],1550149963058,"{'location': [-144.397928, 50.329537], 'depth'...","{'@class': '.XMooring', 'assetId': 160, 'remot...",None,None,None,None,GP03FLMB-RIM01-02-CTDMOG060,"{'@class': '.XInstrument', 'calibration': [{'@...",UNKNOWN,1,4145.0


In [251]:
df3.columns.values

array(['@class', 'assetUid', 'dataSource', 'deployCruiseInfo',
       'deployedBy', 'deploymentNumber', 'editPhase', 'eventId',
       'eventName', 'eventStartTime', 'eventStopTime', 'eventType',
       'inductiveId', 'ingestInfo', 'lastModifiedTimestamp', 'location',
       'mooring', 'node', 'notes', 'recoverCruiseInfo', 'recoveredBy',
       'referenceDesignator', 'sensor', 'tense', 'versionNumber',
       'waterDepth'], dtype=object)

In [252]:
df3['dataSource']

0    Load from [GP03FLMB_Deploy.xlsx]
1    Load from [GP03FLMB_Deploy.xlsx]
2    Load from [GP03FLMB_Deploy.xlsx]
3    Load from [GP03FLMB_Deploy.xlsx]
4    Load from [GP03FLMB_Deploy.xlsx]
5    Load from [GP03FLMB_Deploy.xlsx]
Name: dataSource, dtype: object

In [253]:
df3['sensor']

0    {'@class': '.XInstrument', 'calibration': [{'@...
1    {'@class': '.XInstrument', 'calibration': [{'@...
2    {'@class': '.XInstrument', 'calibration': [{'@...
3    {'@class': '.XInstrument', 'calibration': [{'@...
4    {'@class': '.XInstrument', 'calibration': [{'@...
5    {'@class': '.XInstrument', 'calibration': [{'@...
Name: sensor, dtype: object

In [254]:
df3['sensor'].iloc[0]['calibration']

[{'@class': '.XCalibration',
  'name': 'CC_a0',
  'calData': [{'@class': '.XCalibrationData',
    'value': -0.0001158514,
    'comments': None,
    'assetUid': 'CGINS-CTDMOG-10255',
    'eventId': 43035,
    'eventType': 'CALIBRATION_DATA',
    'eventName': 'CC_a0',
    'eventStartTime': 1504656000000,
    'eventStopTime': None,
    'notes': None,
    'tense': 'UNKNOWN',
    'dataSource': 'CGINS-CTDMOG-10255__20170906_Cal_Info.xlsx',
    'lastModifiedTimestamp': 1550150091025},
   {'@class': '.XCalibrationData',
    'value': -0.0001314068,
    'comments': None,
    'assetUid': 'CGINS-CTDMOG-10255',
    'eventId': 10622,
    'eventType': 'CALIBRATION_DATA',
    'eventName': 'CC_a0',
    'eventStartTime': 1447113600000,
    'eventStopTime': None,
    'notes': None,
    'tense': 'UNKNOWN',
    'dataSource': 'CGINS-CTDMOG-10255__20151110_Cal_Info.xlsx',
    'lastModifiedTimestamp': 1550150025139},
   {'@class': '.XCalibrationData',
    'value': -0.0001546605,
    'comments': None,
    'ass